<img src='logo/dsl-logo.png' width="500" align="center" />

# HR Competition

## Load Data

### Initializations

In [86]:
# Bibliotheken einbinden
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [87]:
# Definition einer Klasse für Text Styles
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Import Data from CSV

In [88]:
# Beschreibung des HR Datensets
with open('data/hr_desc.txt', 'r') as file:
    print (file.read())

Warum verlassen uns unsere besten und erfahrensten Mitarbeiter und Mitarbeiterinnen?

Ziel: Entwicklung eines Prognosemodells für die Vorhersage, ob ein Mitarbeiter oder eine Mitarbeiterin das Unternehmen als nächstes verlassen könnte.

Verfügbare Attribute:
- satisfaction_level: Zufriedenheitslevel (0-1)
- last_evaluation: Zeit in Jahren seit der letzten Evaluierung
- number_project: Anzahl abgeschlossener Projekte 
- average_monthly_hours: Durchschnittliche monatliche Arbeitsstunden 
- time_spent_company: Zeit in Jahren im Unternehmen
- work_accident: Lag ein Arbeitsunfall vor? 
- promotion_last_5years: Gab es eine Beförderung in den letzten fünf Jahren?
- department: Abteilung, für die gearbeitet wird
- salary: relativer Gehaltslevel 
- left: Wurde gekündigt?


In [89]:
# Trainingsdaten einlesen
df = pd.read_csv('data/hr_train.csv', delimiter=';', decimal='.')
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.55,0.86,4,169,6,0,0,0,IT,medium
1,0.66,0.48,4,229,4,0,0,0,sales,medium
2,0.56,0.67,5,165,3,1,0,0,management,medium
3,0.59,1.00,2,155,5,0,1,0,sales,low
4,0.87,0.49,4,149,2,0,0,0,sales,low


In [90]:
# Spaltenbezeichner anpassen
df = df.rename(columns={
    'satisfaction_level': 'satisfactionLevel',
    'last_evaluation': 'yearsSinceEvaluation',
    'number_project': 'numberOfProjects',
    'average_montly_hours': 'averageMonthlyHours',
    'time_spend_company': 'yearsAtCompany',
    'Work_accident': 'workAccident',
    'left': 'hasLeftCompany',
    'promotion_last_5years': 'gotPromotion',
    'department': 'department',
    'salary': 'salary'
})

In [91]:
# Format des Trainingsdaten Dataframes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 10 columns):
satisfactionLevel       11999 non-null float64
yearsSinceEvaluation    11999 non-null float64
numberOfProjects        11999 non-null int64
averageMonthlyHours     11999 non-null int64
yearsAtCompany          11999 non-null int64
workAccident            11999 non-null int64
hasLeftCompany          11999 non-null int64
gotPromotion            11999 non-null int64
department              11999 non-null object
salary                  11999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 937.5+ KB


In [92]:
# Testdaten einlesen
df_test = pd.read_csv('data/hr_test.csv', delimiter=';', decimal='.')
df_test.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary
0,0.10,0.93,7,258,4,0,0,technical,low
1,0.24,0.55,6,231,4,0,0,sales,low
2,0.23,0.84,5,140,4,0,0,IT,low
3,0.42,0.54,2,159,3,0,0,sales,medium
4,0.43,0.47,2,144,3,0,0,hr,medium


In [93]:
# Spaltenbezeichner anpassen
df_test = df_test.rename(columns={
    'satisfaction_level': 'satisfactionLevel',
    'last_evaluation': 'yearsSinceEvaluation',
    'number_project': 'numberOfProjects',
    'average_montly_hours': 'averageMonthlyHours',
    'time_spend_company': 'yearsAtCompany',
    'Work_accident': 'workAccident',
    'promotion_last_5years': 'gotPromotion',
    'department': 'department',
    'salary': 'salary'
})

In [94]:
# Format des Testdaten Dataframes
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
satisfactionLevel       3000 non-null float64
yearsSinceEvaluation    3000 non-null float64
numberOfProjects        3000 non-null int64
averageMonthlyHours     3000 non-null int64
yearsAtCompany          3000 non-null int64
workAccident            3000 non-null int64
gotPromotion            3000 non-null int64
department              3000 non-null object
salary                  3000 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 211.0+ KB


### Check Appropriate Data Types

In [95]:
# Anzahl Unique Values pro Column
print(color.UNDERLINE + color.BOLD + 'Number of Unique Values per Column:' + color.END)
for col in df.columns:
    col_unique_value_count = df[col].unique().size
    print('\t' + col + color.BOLD,':', col_unique_value_count, color.END)

Number of Unique Values per Column:
	satisfactionLevel : 92 
	yearsSinceEvaluation : 65 
	numberOfProjects : 6 
	averageMonthlyHours : 215 
	yearsAtCompany : 8 
	workAccident : 2 
	hasLeftCompany : 2 
	gotPromotion : 2 
	department : 10 
	salary : 3 


In [96]:
df.columns

Index(['satisfactionLevel', 'yearsSinceEvaluation', 'numberOfProjects',
       'averageMonthlyHours', 'yearsAtCompany', 'workAccident',
       'hasLeftCompany', 'gotPromotion', 'department', 'salary'],
      dtype='object')

In [97]:
# Ausgabe der einzelnen Ausprägungen und deren Anzahl pro Spalte mit höchstens 11 Unique Values
for col in df.columns:
    if df[col].unique().size <= 10:
        print()
        print(color.BOLD + ' attr : count for ' + col + color.END)
        uniques, counts = np.unique(df[col], return_counts=True)
        zipped = zip(uniques, counts)
        dictionary = dict(zipped)
        pd.DataFrame.from_dict(data=dictionary, orient='index')
        dictionary
        for unique, count in dictionary.items():
            print('\t', unique, ':', count)


 attr : count for numberOfProjects
	 2 : 1934
	 3 : 3246
	 4 : 3483
	 5 : 2191
	 6 : 938
	 7 : 207

 attr : count for yearsAtCompany
	 2 : 2584
	 3 : 5165
	 4 : 2049
	 5 : 1156
	 6 : 576
	 7 : 163
	 8 : 132
	 10 : 174

 attr : count for workAccident
	 0 : 10245
	 1 : 1754

 attr : count for hasLeftCompany
	 0 : 9149
	 1 : 2850

 attr : count for gotPromotion
	 0 : 11739
	 1 : 260

 attr : count for department
	 IT : 972
	 RandD : 631
	 accounting : 627
	 hr : 576
	 management : 514
	 marketing : 666
	 product_mng : 733
	 sales : 3325
	 support : 1795
	 technical : 2160

 attr : count for salary
	 high : 997
	 low : 5845
	 medium : 5157


In [98]:
# Datentyp für bestimmte Spalten in Category umwandeln
for col in ['workAccident', 'hasLeftCompany', 'gotPromotion', 'department', 'salary']:
    print('transforming', col)
    df[col] = df[col].astype('category')
    if col != 'hasLeftCompany':
        df_test[col] = df_test[col].astype('category')

transforming workAccident
transforming hasLeftCompany
transforming gotPromotion
transforming department
transforming salary


In [99]:
# Ergebnis überprüfen
print(color.BOLD + 'Info about HR Training Dataframe' + color.END)
print(df.info())
print()
print(color.BOLD + 'Info about HR Test Dataframe' + color.END)
print(df_test.info())

Info about HR Training Dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 10 columns):
satisfactionLevel       11999 non-null float64
yearsSinceEvaluation    11999 non-null float64
numberOfProjects        11999 non-null int64
averageMonthlyHours     11999 non-null int64
yearsAtCompany          11999 non-null int64
workAccident            11999 non-null category
hasLeftCompany          11999 non-null category
gotPromotion            11999 non-null category
department              11999 non-null category
salary                  11999 non-null category
dtypes: category(5), float64(2), int64(3)
memory usage: 528.2 KB
None

Info about HR Test Dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
satisfactionLevel       3000 non-null float64
yearsSinceEvaluation    3000 non-null float64
numberOfProjects        3000 non-null int64
averageMonthlyHours     3000 non-null int64
yea

### Export Dataframe for Next Steps

In [100]:
df.to_pickle('exchange/hr_01_loaded_train.pkl')
df_test.to_pickle('exchange/hr_01_loaded_test.pkl')